# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f898c8de1f0>
├── 'a' --> tensor([[-0.0113, -1.1151,  1.1722],
│                   [ 0.7259,  1.1265,  1.6400]])
└── 'x' --> <FastTreeValue 0x7f898c8deac0>
    └── 'c' --> tensor([[-0.9235,  0.0731,  0.0922,  0.9548],
                        [-0.9627, -0.5606, -0.3010, -0.1171],
                        [ 0.6720,  0.7402,  0.6757,  0.2325]])

In [4]:
t.a

tensor([[-0.0113, -1.1151,  1.1722],
        [ 0.7259,  1.1265,  1.6400]])

In [5]:
%timeit t.a

58.6 ns ± 0.0526 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f898c8de1f0>
├── 'a' --> tensor([[-0.5188,  0.7814,  0.8601],
│                   [ 1.4885,  0.1075, -0.3253]])
└── 'x' --> <FastTreeValue 0x7f898c8deac0>
    └── 'c' --> tensor([[-0.9235,  0.0731,  0.0922,  0.9548],
                        [-0.9627, -0.5606, -0.3010, -0.1171],
                        [ 0.6720,  0.7402,  0.6757,  0.2325]])

In [7]:
%timeit t.a = new_value

58 ns ± 0.0231 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0113, -1.1151,  1.1722],
               [ 0.7259,  1.1265,  1.6400]]),
    x: Batch(
           c: tensor([[-0.9235,  0.0731,  0.0922,  0.9548],
                      [-0.9627, -0.5606, -0.3010, -0.1171],
                      [ 0.6720,  0.7402,  0.6757,  0.2325]]),
       ),
)

In [10]:
b.a

tensor([[-0.0113, -1.1151,  1.1722],
        [ 0.7259,  1.1265,  1.6400]])

In [11]:
%timeit b.a

55.9 ns ± 0.0444 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.6222, -0.7740,  0.8048],
               [-0.4885,  1.2544,  0.1452]]),
    x: Batch(
           c: tensor([[-0.9235,  0.0731,  0.0922,  0.9548],
                      [-0.9627, -0.5606, -0.3010, -0.1171],
                      [ 0.6720,  0.7402,  0.6757,  0.2325]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.17 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

841 ns ± 8.84 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 61.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 885 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 262 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f88e0beca00>
├── 'a' --> tensor([[[-0.0113, -1.1151,  1.1722],
│                    [ 0.7259,  1.1265,  1.6400]],
│           
│                   [[-0.0113, -1.1151,  1.1722],
│                    [ 0.7259,  1.1265,  1.6400]],
│           
│                   [[-0.0113, -1.1151,  1.1722],
│                    [ 0.7259,  1.1265,  1.6400]],
│           
│                   [[-0.0113, -1.1151,  1.1722],
│                    [ 0.7259,  1.1265,  1.6400]],
│           
│                   [[-0.0113, -1.1151,  1.1722],
│                    [ 0.7259,  1.1265,  1.6400]],
│           
│                   [[-0.0113, -1.1151,  1.1722],
│                    [ 0.7259,  1.1265,  1.6400]],
│           
│                   [[-0.0113, -1.1151,  1.1722],
│                    [ 0.7259,  1.1265,  1.6400]],
│           
│                   [[-0.0113, -1.1151,  1.1722],
│                    [ 0.7259,  1.1265,  1.6400]]])
└── 'x' --> <FastTreeValue 0x7f88e0becbb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 21.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f88e05b0340>
├── 'a' --> tensor([[-0.0113, -1.1151,  1.1722],
│                   [ 0.7259,  1.1265,  1.6400],
│                   [-0.0113, -1.1151,  1.1722],
│                   [ 0.7259,  1.1265,  1.6400],
│                   [-0.0113, -1.1151,  1.1722],
│                   [ 0.7259,  1.1265,  1.6400],
│                   [-0.0113, -1.1151,  1.1722],
│                   [ 0.7259,  1.1265,  1.6400],
│                   [-0.0113, -1.1151,  1.1722],
│                   [ 0.7259,  1.1265,  1.6400],
│                   [-0.0113, -1.1151,  1.1722],
│                   [ 0.7259,  1.1265,  1.6400],
│                   [-0.0113, -1.1151,  1.1722],
│                   [ 0.7259,  1.1265,  1.6400],
│                   [-0.0113, -1.1151,  1.1722],
│                   [ 0.7259,  1.1265,  1.6400]])
└── 'x' --> <FastTreeValue 0x7f88e05b01c0>
    └── 'c' --> tensor([[-0.9235,  0.0731,  0.0922,  0.9548],
                        [-0.9627, -0.5606, -0.3010, -0.1171],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 758 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.9 µs ± 245 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.9235,  0.0731,  0.0922,  0.9548],
                       [-0.9627, -0.5606, -0.3010, -0.1171],
                       [ 0.6720,  0.7402,  0.6757,  0.2325]],
              
                      [[-0.9235,  0.0731,  0.0922,  0.9548],
                       [-0.9627, -0.5606, -0.3010, -0.1171],
                       [ 0.6720,  0.7402,  0.6757,  0.2325]],
              
                      [[-0.9235,  0.0731,  0.0922,  0.9548],
                       [-0.9627, -0.5606, -0.3010, -0.1171],
                       [ 0.6720,  0.7402,  0.6757,  0.2325]],
              
                      [[-0.9235,  0.0731,  0.0922,  0.9548],
                       [-0.9627, -0.5606, -0.3010, -0.1171],
                       [ 0.6720,  0.7402,  0.6757,  0.2325]],
              
                      [[-0.9235,  0.0731,  0.0922,  0.9548],
                       [-0.9627, -0.5606, -0.3010, -0.1171],
                       [ 0.6720,  0.7402,  0.6757,  0.2325]],

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 260 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.9235,  0.0731,  0.0922,  0.9548],
                      [-0.9627, -0.5606, -0.3010, -0.1171],
                      [ 0.6720,  0.7402,  0.6757,  0.2325],
                      [-0.9235,  0.0731,  0.0922,  0.9548],
                      [-0.9627, -0.5606, -0.3010, -0.1171],
                      [ 0.6720,  0.7402,  0.6757,  0.2325],
                      [-0.9235,  0.0731,  0.0922,  0.9548],
                      [-0.9627, -0.5606, -0.3010, -0.1171],
                      [ 0.6720,  0.7402,  0.6757,  0.2325],
                      [-0.9235,  0.0731,  0.0922,  0.9548],
                      [-0.9627, -0.5606, -0.3010, -0.1171],
                      [ 0.6720,  0.7402,  0.6757,  0.2325],
                      [-0.9235,  0.0731,  0.0922,  0.9548],
                      [-0.9627, -0.5606, -0.3010, -0.1171],
                      [ 0.6720,  0.7402,  0.6757,  0.2325],
                      [-0.9235,  0.0731,  0.0922,  0.9548],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 179 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

318 µs ± 1.57 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
